In [10]:
from loadTDMS import *
from fft_test import *
import numpy as np
# import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
# import seaborn as sns

# 実験日選択

In [9]:
# date = 20211025
# date = 20211115
# date = 20220630
# date = 20221027
# date = 20230113
# date = 20230117
# date = 20230120
date = 20230123

# # cuttingﾌｫﾙﾀﾞの中の実切削中のサンプリング数特定

In [5]:
num = 3
path = rf"Z:\100 Raw_data\{date}---切削実験\Dynamometer\Cutting\Dynamo_{num}.tdms"
npath = rf"Z:\100 Raw_data\{date}---切削実験\Dynamometer\Non-Cutting\Dynamo_{num}.tdms"

cd = load_tdms(path, ch_num=3)
ncd = load_tdms(npath, ch_num=3)

print(len(cd[1]))
print(len(ncd[1]))

10460356
652682


In [6]:
%matplotlib qt
plt.figure()
plt.plot(cd[1])

## 日付 : 切削中のサンプル数 / last path 
- 20211025 : 5060000 / 24path
- 20211115 : 5010000 / 238path
- 20220630 : 4670000 / 735path
- 20221027 : 4930000 / 168path
- 20230113 : 9970000 / 120path
- 20230117 : 9970000 / 340path
- 20230120 : 9970000 / 460path
- 20230123 : [100000:9600000] / 460path

In [12]:
if date == 20211025:
    last_path = 24
    last_sample_num = 5060000
if date == 20211115:
    last_path = 238
    last_sample_num = 5010000
if date == 20220630:
    last_path = 735
    last_sample_num = 4670000 
if date == 20221027:
    last_path = 168
    last_sample_num = 4930000
    
if date == 20230113:
    last_path = 120
    last_sample_num = 9970000

if date == 20230117:
    last_path = 460
    last_sample_num = 9970000
    
if date == 20230120:
    last_path = 460
    last_sample_num = 9970000

if date == 20230123:
    last_path = 460
    last_sample_num = 9960000    
    
    print(rf"date={date}, last_path={last_path}, last_sample_num={last_sample_num}")

date=20230123, last_path=460, last_sample_num=9960000


## get cutting voltage avg
cuttingの平均電圧値の推移調べ

In [13]:
cutting_avg = np.array([])
for num in tqdm(range(1,last_path+1)):
    path = rf"Z:\100 Raw_data\{date}---切削実験\Dynamometer\Cutting\Dynamo_{num}.tdms"
    cd = load_tdms(path, ch_num=3)
    a = np.array(cd[1][100000:last_sample_num])
    cutting_avg = np.append(cutting_avg, np.average(a))
    
np.save(rf"Z:\200 Produced_data\{date}\CF\cutting_avg",cutting_avg)
cutting_avg = np.load(rf"Z:\200 Produced_data\{date}\CF\cutting_avg.npy")

100%|██████████| 460/460 [12:44<00:00,  1.66s/it]


In [14]:
%matplotlib qt
cutting_avg = np.load(rf"Z:\200 Produced_data\{date}\CF\cutting_avg.npy")
plt.plot(cutting_avg)
plt.savefig(rf"Z:\200 Produced_data\{date}\CF\cutting_avg.png")

## get non cutting voltage avg
non-cuttingの平均電圧値の推移調べ

In [15]:
non_cutting_avg = np.array([])
for num in tqdm(range(1,last_path+1)):
    path = rf"Z:\100 Raw_data\{date}---切削実験\Dynamometer\Non-Cutting\Dynamo_{num}.tdms"
    ncd = load_tdms(path, ch_num=3)
    a = np.array(ncd[1])
    non_cutting_avg = np.append(non_cutting_avg, np.average(a))

np.save(rf"Z:\200 Produced_data\{date}\CF\non_cutting_avg",non_cutting_avg)

100%|██████████| 460/460 [01:09<00:00,  6.63it/s]


In [16]:
%matplotlib qt
non_cutting_avg = np.load(rf"Z:\200 Produced_data\{date}\CF\non_cutting_avg.npy")
plt.plot(non_cutting_avg)
plt.savefig(rf"Z:\200 Produced_data\{date}\CF\non_cutting_avg.png")

## 発散除去工程

### 2023 0113
non-cuttingの平均電圧は、発散しているpathを除けば、おおよそ右肩下がりだったので  
1path目(最初)と120path目(最後)を結ぶ1次関数に置き換えて、new_non_cutting_avgを算出した。

In [32]:
%matplotlib qt

date = 20230113
non_cutting_avg = np.load(rf"Z:\200 Produced_data\{date}\CF\non_cutting_avg.npy")

katamuki = (non_cutting_avg[-1] - non_cutting_avg[0]) / len(non_cutting_avg)
y_seppen = non_cutting_avg[-1] - (katamuki * 120)

print(katamuki)
print(y_seppen)

new_non_cutting_avg = np.array([])
for i in range(1,121):
    new_non_cutting_avg = np.append(new_non_cutting_avg, katamuki * (i+1) + y_seppen)
np.save(rf"Z:\200 Produced_data\{date}\CF\new_non_cutting_avg", new_non_cutting_avg)

non_cutting_avg = np.load(rf"Z:\200 Produced_data\{date}\CF\non_cutting_avg.npy")
new_non_cutting_avg = np.load(rf"Z:\200 Produced_data\{date}\CF\new_non_cutting_avg.npy")

plt.plot(non_cutting_avg)
plt.plot(new_non_cutting_avg)

-0.0005201947914126019
-0.00602323142811656


### 2023 0117
non-cuttingの平均電圧は、発散しているpathを除いても、1次関数の直線には近似できなさそうだったので  
発散しているpathの平均電圧に関しては、そのpathを中心として、自分を除く前後それぞれ12pathずつ、  
合計24path分の平均値に置き換えて、new_non_cutting_avgを算出した。

In [43]:
date = 20230117
non_cutting_avg = np.load(rf"Z:\200 Produced_data\{date}\CF\non_cutting_avg.npy")

ncd = np.where(non_cutting_avg < -0.8, 0, non_cutting_avg)
ncd_origin = non_cutting_avg.copy()

index = np.where(ncd == 0)
for i in index[0]:
    ncd[i] = np.average(ncd[i-12:i+12])
# plt.plot(nnd_origin)
# plt.plot(nnd)

np.save(rf"Z:\200 Produced_data\{date}\CF\new_non_cutting_avg",ncd)

non_cutting_avg = np.load(rf"Z:\200 Produced_data\{date}\CF\non_cutting_avg.npy")
new_non_cutting_avg = np.load(rf"Z:\200 Produced_data\{date}\CF\new_non_cutting_avg.npy")

plt.plot(non_cutting_avg)
plt.plot(new_non_cutting_avg)

### 2023 0120
non-cuttingの平均電圧は、発散しているpathを除けば、おおよそ右肩下がりだったので  
1path目(最初)と460path目(最後)を結ぶ1次関数に置き換えて、new_non_cutting_avgを算出した。

In [38]:
date = 20230120
non_cutting_avg = np.load(rf"Z:\200 Produced_data\{date}\CF\non_cutting_avg.npy")

katamuki = (non_cutting_avg[-1] - non_cutting_avg[0]) / len(non_cutting_avg)
y_seppen = non_cutting_avg[-1] - (katamuki * 460)

print(katamuki)
print(y_seppen)

new_non_cutting_avg = np.array([])
for i in range(1,460):
    new_non_cutting_avg = np.append(new_non_cutting_avg, katamuki * (i+1) + y_seppen)
np.save(rf"Z:\200 Produced_data\{date}\CF\new_non_cutting_avg", new_non_cutting_avg)

non_cutting_avg = np.load(rf"Z:\200 Produced_data\{date}\CF\non_cutting_avg.npy")
new_non_cutting_avg = np.load(rf"Z:\200 Produced_data\{date}\CF\new_non_cutting_avg.npy")

plt.plot(non_cutting_avg)
plt.plot(new_non_cutting_avg)

-0.00028426609933376314
0.03528834879398346
